In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime

In [2]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
data, targets = load_boston(True)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
targets = targets.reshape(-1,1)
print(data.shape)
print(targets.shape)

(506, 13)
(506, 1)


In [6]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_dir = "tf_logs_linear_regression"
log_dir = "{}/run-{}".format(root_dir,now)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data, targets, test_size = 0.2, random_state=42)

In [8]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[None,13], name="X")
y = tf.placeholder(dtype=tf.float32, shape=[None,1], name="y")
W = tf.Variable(tf.random_uniform([13,1]), name="W")
b = tf.Variable(tf.random_uniform([1]), name="b")

y_pred = tf.matmul(X,W) + b

with tf.name_scope("loss") as scope:
    error = y_pred - y
    loss = tf.reduce_mean(tf.square(error), name="mse")
    
training_op = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(loss)
loss_summary = tf.summary.scalar("MSE", loss)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [11]:
n_epochs = 2000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        epoch_loss, _ = sess.run([loss,training_op], feed_dict={X:X_train, y: y_train})
        if epoch % 100 == 0:
            mse_loss = sess.run(loss_summary, feed_dict={X:X_train, y: y_train})
            file_writer.add_summary(mse_loss, epoch)
            print("Epoch {}: Loss :{}".format(epoch, epoch_loss))
    
    print("Training Complete")
    y_test_pred = sess.run(y_pred, feed_dict={X:X_test})
    diff = y_test_pred.flatten() - y_test.flatten()
    abs_diff = [abs(x) for x in diff]
    
file_writer.flush()
file_writer.close()

Epoch 0: Loss :599.7053833007812
Epoch 100: Loss :21.922100067138672
Epoch 200: Loss :21.697118759155273
Epoch 300: Loss :21.656766891479492
Epoch 400: Loss :21.645822525024414
Epoch 500: Loss :21.642684936523438
Epoch 600: Loss :21.64177894592285
Epoch 700: Loss :21.641517639160156
Epoch 800: Loss :21.641441345214844
Epoch 900: Loss :21.641420364379883
Epoch 1000: Loss :21.641414642333984
Epoch 1100: Loss :21.641414642333984
Epoch 1200: Loss :21.64141273498535
Epoch 1300: Loss :21.641414642333984
Epoch 1400: Loss :21.64141273498535
Epoch 1500: Loss :21.641414642333984
Epoch 1600: Loss :21.64141273498535
Epoch 1700: Loss :21.64141082763672
Epoch 1800: Loss :21.641414642333984
Epoch 1900: Loss :21.64141082763672
Training Complete
